In [1]:
import os, sys, datetime

from alphahelix_database_tools.cloud_database import CloudArticlesDatabase

article_db  = CloudArticlesDatabase("/Users/yahoo168/Desktop/資料庫_測試功能/MDB_config")
# article_db.save_shorts()
# article_db.save_shorts_summary()

In [2]:
stock_ticker_list = ["AAPL", "GOOG", "MSFT", "LAZR", "NVTS", "QCOM", "TSLA", "NET", "ON", "OXY", "TSM"]
bond_ticker_list = ["RITM", "CXW", "F", "MO", "BA"]
pool_ticker_list = stock_ticker_list + bond_ticker_list
pool_ticker_list.sort()

# for ticker in pool_ticker_list:
#     article_db.save_shorts_summary()
    # article_db.save_stock_report_review(ticker=ticker)
    
# 定义开始和结束日期
start_date = datetime.datetime(2024, 6, 16)
end_date = datetime.datetime(2024, 7, 11)

# 生成从开始日期到结束日期的日期列表
date_list = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]
for date in date_list:
    article_db.save_shorts_summary(end_date=date, ticker_list=pool_ticker_list)

[20240711 23:37:38 INFO    ] [SHORTS][SUMMARY]2024-06-15-2024-06-16
[20240711 23:37:42 INFO    ] [MDB][server][check] connected to the 'articles' cluster
[20240711 23:37:42 INFO    ] [GPT][shorts_summary][AAPL] 共48則shorts，預計摘要字數約384
[20240711 23:37:52 INFO    ] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[20240711 23:37:52 INFO    ] [GPT][shorts_summary][BA][no shorts exist in the selected period]
[20240711 23:37:53 INFO    ] [GPT][shorts_summary][CXW][no shorts exist in the selected period]
[20240711 23:37:53 INFO    ] [GPT][shorts_summary][F] 共3則shorts，預計摘要字數約24
[20240711 23:37:55 INFO    ] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[20240711 23:37:55 INFO    ] [GPT][shorts_summary][GOOG] 共30則shorts，預計摘要字數約240
[20240711 23:38:05 INFO    ] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[20240711 23:38:05 INFO    ] [GPT][shorts_summary][LAZR][no shorts exist in the selected period]

In [9]:
article_db.process_raw_stock_report()

[20240711 14:54:03 INFO    ] [SERVER][Data Process][共180篇non auto stock report待處理]
[20240711 14:54:03 INFO    ] [SERVER][Data Process][開始處理第0篇non auto stock report]
[nltk_data] Downloading package punkt to /Users/yahoo168/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[20240711 14:54:05 INFO    ] [LLM][preprocess]Barclays_Apple_Inc_AAPL_Preview_Expecting_In_line_March_Q_and_Potential_June_Q_Guide_down_on_Weak_Hardware_iPhone_Demand.pdf
[20240711 14:54:16 INFO    ] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[20240711 14:54:22 INFO    ] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[20240711 14:54:22 INFO    ] [SERVER][Data Process][開始處理第1篇non auto stock report]
[nltk_data] Downloading package punkt to /Users/yahoo168/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[20240711 14:54:23 INFO    ] [LLM][preprocess]Barclays_Apple_Inc_AAPL_Supplier_Tracker_Lower_Hardware_Builds_for_Feb_Month

In [ ]:
ticker = "MSFT"

stock_following_issue_meta_list = article_db.MDB_client["users"]["following_issues"].find({"tickers": ticker}, limit=10)
stock_following_issue_list = [meta["issue"] for meta in stock_following_issue_meta_list]
stock_following_issue_list

In [ ]:
processed_list = list(article_db.MDB_client["preprocessed_content"]["stock_report"].find())

In [ ]:
for meta in processed_list:
    blob_name = meta["blob_name"]
    raw_meta = article_db.MDB_client["raw_content"]["raw_stock_report_non_auto"].update_one({"blob_name": blob_name}, {"$set": {"is_processed": True}})
    print(raw_meta)
    

In [ ]:
# 查询条件：查找所有不包含is_processed字段的文档
query = {"is_processed": {"$exists": False}}

# 更新操作：将is_processed字段设置为false
update = {"$set": {"is_processed": False}}

# 批量更新文档
result = article_db.MDB_client["raw_content"]["raw_stock_report_non_auto"].update_many(query, update)

In [10]:
col = article_db.MDB_client["preprocessed_content"]["shorts_summary"]

update = {"$rename": {"date": "processed_timestamp"}}
# 批量更新文档
result = col.update_many({}, update)

In [6]:
unprocessed_stock_report_meta_list = list(article_db.MDB_client["raw_content"]["raw_stock_report_non_auto"].find({"is_processed": False, ""}))
len(unprocessed_stock_report_meta_list)

180